In [2]:
ingredients = {
    "Rice": "1 kg",
    "Wheat Flour": "2 kg",
    "Sugar": "1 kg",
    "Salt": "500 g",
    "Sunflower Oil": "1 L",
    "Pasta": "500 g",
    "Butter": "250 g",
    "Eggs": "12 pcs",
    "Milk": "1 L",
    "Potatoes": "1 kg",
    "Onions": "500 g",
    "Garlic": "200 g",
    "Tomatoes": "500 g",
    "Black Pepper": "100 g",
    "Turmeric": "100 g",
    "Cumin Seeds": "100 g"
}
def ingredients_to_sentence(ingredients: dict) -> str:
    parts = []
    for item, qty in ingredients.items():
        parts.append(f"{qty} of {item}")
    return ", ".join(parts[:-1]) + " and " + parts[-1] + "."
Available_ingredients = ingredients_to_sentence(ingredients)
print("Available ingredients:", Available_ingredients)

Available ingredients: 1 kg of Rice, 2 kg of Wheat Flour, 1 kg of Sugar, 500 g of Salt, 1 L of Sunflower Oil, 500 g of Pasta, 250 g of Butter, 12 pcs of Eggs, 1 L of Milk, 1 kg of Potatoes, 500 g of Onions, 200 g of Garlic, 500 g of Tomatoes, 100 g of Black Pepper, 100 g of Turmeric and 100 g of Cumin Seeds.


In [4]:
dietary_restrictions = [
    "Vegetarian",
    "Vegan",
    "Gluten-free",
    "Lactose-free",
    "Nut-free",
    "Soy-free",
    "Sugar-free",
    "Keto-friendly",
    "Paleo",
    "Low-sodium",
    "Low-carb",
    "Halal",
    "Kosher"
]
def dietary_restrictions_to_sentence(restrictions: list) -> str:
    if not restrictions:
        return "No dietary restrictions."
    if len(restrictions) == 1:
        return f"Available dietary restriction: {restrictions[0]}."
    return "Available dietary restrictions: " + ", ".join(restrictions[:-1]) + " and " + restrictions[-1] + "." 




In [11]:
recipe_prompt_structure = {
    "ingredients": "",
    "dietary_restrictions": "",
    "cuisine": "",
    "difficulty": "",
    "servings": "",
    "cooking_time_minutes": "",    
}
recipe_prompt = {
    "ingredients": ingredients_to_sentence(ingredients),
    "dietary_restrictions": dietary_restrictions_to_sentence(dietary_restrictions),
    "cuisine": "South Indian",
    "difficulty": "Medium",
    "servings": "2",
    "cooking_time_minutes": "50",    
}

In [ ]:
from pydantic import BaseModel
from typing import List, Optional, Union


class PropertyValue(BaseModel):
    name: str
    value: Union[str, int, float]
    unitCode: Optional[str] = None
    # "@type" omitted because it's constant in your data


class NutritionInformation(BaseModel):
    calories: str
    fatContent: str


class Recipe(BaseModel):
    context: str
    type: str
    author: str
    cookTime: str
    datePublished: str
    description: str
    image: str
    recipeIngredient: List[Union[str, PropertyValue]]
    name: str
    nutrition: NutritionInformation
    prepTime: str
    recipeInstructions: str
    recipeYield: str
    suitableForDiet: Optional[str]





In [55]:
from langchain.prompts import PromptTemplate

recipe_generate_prompt = PromptTemplate(
    input_variables=list(recipe_prompt.keys()),
    template="""
You are a world-class chef, culinary scientist, and nutrition expert.

Your task is to create a recipe in JSON-LD format following the schema.org Recipe specification.
The output structure will be enforced automatically, so do not include extra text, markdown, or explanations.

Use the following configuration to generate recipe content:

Ingredients (with portions when available):
{ingredients}

Dietary Restrictions (if any):
{dietary_restrictions}

Cuisine Style:
{cuisine}

Difficulty Level:
{difficulty}

Servings:
{servings}

Estimated Cooking Time (minutes):
{cooking_time_minutes}

Guidelines:

- Match ingredient amounts realistically to the recipe.
- Convert total cooking time and prep time into ISO 8601 duration strings, e.g., PT50M.
- Provide step-by-step instructions as an array of short, clear steps.
- Include a short but appealing description of the dish.
- Choose a recipe name that reflects the cuisine and preparation style.
- Choose an author name appropriate for a professional recipe.
- For suitableForDiet: Map dietary restrictions when applicable. Examples:
  - Vegetarian → "https://schema.org/VegetarianDiet"
  - Vegan → "https://schema.org/VeganDiet"
  - Gluten-free → "https://schema.org/GlutenFreeDiet"
  If dietary restrictions do not match known values, leave it an empty string.
- Nutrition values should be reasonable estimates based on typical servings.
- This is a structured output. Return only valid JSON-LD matching the enforced schema.
"""
)


In [49]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.ai.generativelanguage_v1beta.types import Tool as GenAITool

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=Google_API_Key,
    temperature=0.1,
    max_retries=3,
)
structured_llm = llm.with_structured_output(
    schema=Recipe.model_json_schema()
)



E0000 00:00:1763406267.638366    5692 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [54]:
Recipe_generator_chain = recipe_generate_prompt | structured_llm
response = Recipe_generator_chain.invoke(
    {
        **{key: recipe_prompt[key] for key in recipe_prompt_structure.keys()}
    },
    tools=[GenAITool(google_search={})],
)



Key '$defs' is not supported in schema, ignoring
Key 'parameters' is not supported in schema, ignoring


In [56]:
response

[{'args': {'description': 'A flavorful and aromatic South Indian curry made with tender potatoes, ripe tomatoes, and a blend of traditional spices. Perfect when served with steamed rice.',
   'nutrition': {'calories': '450 calories',
    '@type': 'NutritionInformation',
    'fatContent': '15 g'},
   'prepTime': 'PT15M',
   'suitableForDiet': 'https://schema.org/VegetarianDiet',
   'recipeIngredient': [{}, {}, {}, {}, {}, {}, {}, {}, {}, {}],
   '@context': 'https://schema.org',
   'recipeCuisine': 'South Indian',
   'name': 'South Indian Potato Curry',
   'image': 'image_of_south_indian_potato_curry.jpg',
   'cookTime': 'PT35M',
   'recipeInstructions': ['Wash, boil, and peel the potatoes. Cut them into medium-sized cubes.',
    'Heat the sunflower oil in a pan over medium heat. Add the cumin seeds and let them splutter.',
    'Add the finely chopped onions and minced garlic. Sauté until the onions turn translucent and golden.',
    'Add the chopped tomatoes and cook until they become 

In [57]:
recipe_data = response[0]['args']

# Filter out empty dictionaries from recipeIngredient
if 'recipeIngredient' in recipe_data:
	recipe_data['recipeIngredient'] = [
		ingredient for ingredient in recipe_data['recipeIngredient']
		if ingredient and (isinstance(ingredient, str) or (isinstance(ingredient, dict) and ingredient))
	]

recipe_obj = Recipe(**recipe_data)
# recipe_obj.name Filter out empty dictionaries from recipeIngredient
if 'recipeIngredient' in recipe_data:
	recipe_data['recipeIngredient'] = [
		ingredient for ingredient in recipe_data['recipeIngredient']
		if ingredient and (isinstance(ingredient, str) or (isinstance(ingredient, dict) and ingredient))
	]

recipe_obj = Recipe(**recipe_data)
# recipe_obj.name Filter out empty dictionaries from recipeIngredient
if 'recipeIngredient' in recipe_data:
	recipe_data['recipeIngredient'] = [
		ingredient for ingredient in recipe_data['recipeIngredient']
		if ingredient and (isinstance(ingredient, str) or (isinstance(ingredient, dict) and ingredient))
	]

recipe_obj = Recipe(**recipe_data)
recipe_obj.name

'South Indian Potato Curry'